# Using native Python

## Default session

When rgxlog is loaded, a default session is created behind the scenes. This is the session that %%spanner uses.
However, one can also use his own session.

Using a session manually enables one to dynamically generate queries, facts, and rules

In [1]:
%load_ext rgxlog

In [2]:
from rgxlog import Session

some_session = Session()

In [3]:
result = some_session.run_query('''
    new uncle(str, str)
    uncle("benjen", "jon")''')

In [4]:
for maybe_uncle in ['ned','robb','benjen']:
    result = some_session.run_query(f'?uncle("{maybe_uncle}",Y)')

printing results for query 'uncle("ned", Y)':
[]

printing results for query 'uncle("robb", Y)':
[]

printing results for query 'uncle("benjen", Y)':
  Y
-----
 jon



# Changing the session of the magic cells

In cases where you want to work with a custom session, but still make use of the magic system, you can overide the session used by the magic system

In [5]:
import rgxlog  # default session starts here
from rgxlog import Session

another_session=Session()

In [6]:
%%spanner
# still using the default session
# TODO@niv: dean, why do the sessions share their execution? is this intentional?
new uncle(str, str)
uncle("bob", "greg")
?uncle(X,Y)

printing results for query 'uncle(X, Y)':
   X    |  Y
--------+------
  bob   | greg
 benjen | jon



In [7]:
default_session = rgxlog.magic_session  # save the default for later
rgxlog.magic_session = another_session
print(default_session._term_graph)
print(another_session._term_graph)

(0) (computed) root
  (1) (computed) relation_declaration: uncle(str, str)
  (2) (computed) add_fact: uncle("bob", "greg")
  (3) (computed) query: uncle(X, Y)

(0) (not_computed) root



In [8]:
%%spanner
# using 'another_session', not the default

new uncle(str, str)
uncle("ross", "rachel")
?uncle(X,Y)

printing results for query 'uncle(X, Y)':
   X    |   Y
--------+--------
  ross  | rachel
  bob   |  greg
 benjen |  jon



In [9]:
rgxlog.magic_session = default_session

In [10]:
%%spanner
# back to using the default session
?uncle(X,Y)

printing results for query 'uncle(X, Y)':
   X    |   Y
--------+--------
  ross  | rachel
  bob   |  greg
 benjen |  jon



# Mixing magics with dynamic session calls

Lets take the GPA example from the introductory tutorial.
What if we want to have multiple rules each looking for GPAs of students in different classes.
We wouldnt want to manually write a rule for every single subject.

In [11]:
from rgxlog.stdlib.python_regex import PYRGX_STRING

session=rgxlog.magic_session = Session()
session.register(**PYRGX_STRING)

We write our data manually. In the future we would be able to import it from csvs/dataframes

In [12]:
%%spanner
new lecturer(str, str)
lecturer("walter", "chemistry")
lecturer("linus", "operation_systems")
lecturer("rick", "physics")

new enrolled(str, str)
enrolled("abigail", "chemistry")
enrolled("abigail", "operation_systems")
enrolled("jordan", "chemistry")
enrolled("gale", "operation_systems")
enrolled("howard", "chemistry")
enrolled("howard", "physics")



gpa_str = "abigail 100 jordan 80 gale 79 howard 60"


In [13]:
%%spanner

gpa(Student,Grade) <- py_rgx_string(gpa_str, "(\w+).*?(\d+)")->(Student, Grade),enrolled(Student,X)

?gpa(X,Y)

printing results for query 'gpa(X, Y)':
    X    |   Y
---------+-----
 howard  |  60
  gale   |  79
 jordan  |  80
 abigail | 100



Now we are going to define the rules using a for loop

In [14]:
subjects=[
    "chemistry",
    "physics",
    "operation_systems",
    "magic",
]

for subject in subjects:
    rule=f"""
    gpa_of_{subject}_students(Student, Grade) <- gpa(Student, Grade), enrolled(Student, "{subject}")
    """
    session.run_query(rule)
    print(rule) # we print the rule here to show you what strings are sent to the session


    gpa_of_chemistry_students(Student, Grade) <- gpa(Student, Grade), enrolled(Student, "chemistry")
    

    gpa_of_physics_students(Student, Grade) <- gpa(Student, Grade), enrolled(Student, "physics")
    

    gpa_of_operation_systems_students(Student, Grade) <- gpa(Student, Grade), enrolled(Student, "operation_systems")
    

    gpa_of_magic_students(Student, Grade) <- gpa(Student, Grade), enrolled(Student, "magic")
    


As you can see, we can use the dynamically defined rules in a magic cell

In [15]:
%%spanner
?gpa_of_operation_systems_students(X,Y)

printing results for query 'gpa_of_operation_systems_students(X, Y)':
    X    |   Y
---------+-----
  gale   |  79
 abigail | 100



And we can also query dynamically

In [16]:
subjects=[
    "chemistry",
    "physics",
    "operation_systems",
    "magic",
]

for subject in subjects:
    query=f"""
    ?gpa_of_{subject}_students(Student, Grade)
    """
    session.run_query(query)

printing results for query 'gpa_of_chemistry_students(Student, Grade)':
  Student  |   Grade
-----------+---------
  howard   |      60
  jordan   |      80
  abigail  |     100

printing results for query 'gpa_of_physics_students(Student, Grade)':
  Student  |   Grade
-----------+---------
  howard   |      60

printing results for query 'gpa_of_operation_systems_students(Student, Grade)':
  Student  |   Grade
-----------+---------
   gale    |      79
  abigail  |     100

printing results for query 'gpa_of_magic_students(Student, Grade)':
[]



# Processing the result of a query in python and using the result in a new query

In [17]:
from rgxlog import Session
from rgxlog.engine.session import format_query_results

In [18]:
session = Session()
result = session.run_query(f'''
    new friends(str, str, str)
    friends("bob", "greg", "clyde")
    friends("steven", "benny", "horace")
    friends("lenny", "homer", "toby")
    ?friends(X,Y,Z)''')
# Notice the default port is taken by the default session, hence the next port is used for the user's session


# now we'll showcase processing the result with native python...
# TODO: this will look less ugly after format_query_results is refactored
_, res, _ = format_query_results(*result[-1])

# lets filter our tuples with some predicate:
filtered = tuple(filter(lambda friends: 'bob' in friends or 'lenny' in friends, res))

# and feed the matching tuples into a new query:
session.run_query('new buddies(str, str)')

for first, second, _ in filtered:
    session.run_query(f'buddies("{first}", "{second}")')

result = session.run_query("?buddies(First, Second)")

printing results for query 'friends(X, Y, Z)':
   X    |   Y   |   Z
--------+-------+--------
 lenny  | homer |  toby
 steven | benny | horace
  bob   | greg  | clyde

printing results for query 'buddies(First, Second)':
  First  |  Second
---------+----------
   bob   |   greg
  lenny  |  homer



# Developing and using your own IE functions

To make your own ie function, you need to implement the input format, output format, and the function itself:

* `ie_function`: a function which yields an iterable of primitive types
* `ie_function_name`: the name by which the function is called inside rgxlog
* `in_rel`: the input types, e.g. [DataTypes.string]
* `out_rel`: the output types, can be either a method which expects an arity, or an iterable, e.g. [DataTypes.span]

now, just register it:
```python
session.register(ie_function, ie_function_name, in_rel, out_rel)
```

In [19]:
# TODO: we already showed how to do this with `get_happy`. can this be deleted?
# example:

# import spacy
# sp = spacy.load('en_core_web_sm')

# from rgxlog.engine.datatypes.primitive_types import DataTypes
# from rgxlog.engine.ie_functions.ie_function_base import IEFunctionData

# class Entities(IEFunctionData):
#     def __init__(self):
#         super().__init__()

#     @staticmethod
#     def ie_function(text):  # this is where you implement the actual processing
#         entities = sp(text).ents
#         return ((entity.text, spacy.explain(entity.label_)) for entity in entities)

#     @staticmethod
#     def get_input_types():  # this is where you define the inputs
#         return [DataTypes.string]

#     @staticmethod
#     def get_output_types(output_arity):  # and this is where you define your outputs
#         return tuple([DataTypes.string] * 2)
